Functions cluster_fun, mapper_pca_func written by Wako Bungula<br>
Time list code and stratum dictionaries written by Wako Bungula<br>
Function mapper - modified from mapper_pca_func, written by Killian Davis<br>
Coloring nodes code was written by Killian Davis<br>
JSON and pickle creation written by Forrest Miller

In [3]:
import kmapper as km
from kmapper.plotlyviz import *

In [4]:
import sklearn
#from sklearn import ensemble
# DBSCAN from sklearn for clustering algorithms
from sklearn.cluster import DBSCAN
# PCA from sklearn for projection/lens creation
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler
# from sklearn.manifold import MDS

# scipy for interpolation
# import scipy 
# from scipy.interpolate import *

In [5]:
# Plotly and Dash
import plotly.graph_objs as go
from kmapper.plotlyviz import plotlyviz
from kmapper.plotlyviz import *
from ipywidgets import (HBox, VBox)
import dash_html_components as html
import dash_core_components as dcc
import dash
from ipywidgets import interactive, HBox, VBox, widgets, interact
import warnings
warnings.filterwarnings("ignore")

In [6]:
import pandas as pd
predicted_df = pd.read_csv(r"C:\Users\killiad\Documents\Senior\REU\Data\water_full.csv")
predicted_df = predicted_df[predicted_df['FLDNUM']=='Havana, IL']
predicted_df["MONTH"] = pd.DatetimeIndex(predicted_df["DATE"]).month
predicted_df["SEASON"] = predicted_df["MONTH"]
seasons = {3 : 'SPRING',
           4 : 'SPRING',
           5 : 'SPRING',
           6 : 'SUMMER',
           7 : 'SUMMER',
           8 : 'SUMMER',
           9 : 'FALL',
           10 : 'FALL',
           11: 'FALL',
           12: 'WINTER',
           1: 'WINTER',
           2: 'WINTER'}
predicted_df = predicted_df.replace({"SEASON" : seasons})

stratum = {'Main channel' : 1, 'Side channel' : 2, 'Backwater area contiguous to the main channel' : 3}
predicted_df = predicted_df.replace({"STRATUM" : stratum})

In [8]:
to_dummy_list = ['STRATUM', 'FLDNUM', 'YEAR']

for var in to_dummy_list:
    temp = pd.get_dummies(predicted_df[var])
    predicted_df = pd.concat([predicted_df, temp], axis = 1)

In [9]:
### Creating three main time spans and two overlapping time spans,
### a total of five time spans

# defining different time periods
# first decade
time_dec1 = [1993, 1994, 1995, 1997, 1998, 1999, 2000]
# second decade
time_dec2 = [2001, 2002,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]
# third decade
time_dec3 = [2014, 2015, 2016, 2017, 2018, 2019, 2020]
# overlap time periods for continuity
time_overlap1 = [1998, 1999, 2000, 2001, 2002,2003, 2004]
time_overlap2 = [2010, 2011, 2012, 2013, 2014, 2015, 2016]
time_list = [time_dec1, time_overlap1, time_dec2, time_overlap2, time_dec3]

time_list_names = ['93-00', '98-04', '01-13', '10-16', '14-20']
time_list =[time_dec1, time_overlap1, time_dec2, time_overlap2, time_dec3]
stratum_list = [1, 2, 3, 4, 5, 6, 7, 9]
Season_names = ["SPRING", "SUMMER", "FALL", "WINTER"]

nonempty_stratum = []
for i in stratum_list:
    if predicted_df[predicted_df['STRATUM'] == i].shape[0]!=0:
        nonempty_stratum.append(i)
print(nonempty_stratum) 


df_stratum_season_time_dict_list = []
df_stratum_season_time_dict = {}
s=""
for i in range(len(time_list)):
    for j in nonempty_stratum:
        for k in Season_names:
            s=str("Stratum ") + str(j)+ " " + k + " " + time_list_names[i] + str(": ")
            df_stratum_season_time_dict[s] = predicted_df[(predicted_df['YEAR'].isin(time_list[i])) &
                                                             (predicted_df['STRATUM'].isin([j])) & 
                                                             (predicted_df['SEASON'].isin([k]))]
            df_stratum_season_time_dict_list.append(df_stratum_season_time_dict)
            df_stratum_season_time_dict = {}
            s=""
print(len(df_stratum_season_time_dict_list))

[1, 2, 3]
60


In [10]:
def cluster_fun(dict_df, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = [7,7], PERC_OVERLAP = [.5,.5]):
    """

    """
    
    
    keys = list(dict_df.keys())
    print(keys)
    X = dict_df.get(keys[0])
    X = X[["WDP", "SECCHI", "TEMP", "DO", 
           "TURB", "VEL", "TP", 
           "TN", "SS", "CHLcal"]]
    if X.shape[0]<DBSCAN_MIN_SAMPLES:
        #print(X)
        print("Not enough data to cluster in ", keys, "_size = ", X.shape[0])
        print("DBSCAN_MIN_SAMPLES", DBSCAN_MIN_SAMPLES)
        return([DBSCAN_MIN_SAMPLES, X.shape[0]])
    
    
    db = DBSCAN(eps=20, min_samples=2).fit(X)
    # Number of clusters in labels, ignoring noise if present.
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    
    return(db)

    
    
def mapper_pca_func(dict_df, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = [7,7], PERC_OVERLAP = [.5,.5]):
    """
    str(df_stratum_season_time_dict_list[1].keys())
    """

    if len(dict_df) == 0:
        print("Not Enough Data")
        return
    
    # 
    keys = list(dict_df.keys())
    print(keys)
    
    X = dict_df.get(keys[0])
    X = X[["WDP", "SECCHI", "TEMP", "DO", 
           "TURB", "VEL", "TP", 
           "TN", "SS", "CHLcal"]]
    
    continuous_variables = ["WDP", "SECCHI", "TEMP", "DO", 
           "TURB", "VEL", "TP", 
           "TN", "SS", "CHLcal"]
    var_to_index = {continuous_variables[i] : i for i in range(len(continuous_variables))}
    
    projected_vars = continuous_variables
    #projected_vars = ['PredictedTP, PredictedTN']
    projected_var_indices = [var_to_index[var] for var in projected_vars]
    
    if X.shape[0]<10:
        #print(X)
        print("Not enough data in ", keys, "_size = ", X.shape[0])
        return(X.shape[0], -1)
    
    X.reset_index(drop = True, inplace = True)

    # defining clustering and kmapper parameters
    DBSCAN_EPSILON = DBSCAN_EPSILON
    DBSCAN_MIN_SAMPLES = DBSCAN_MIN_SAMPLES
    N_CUBES = N_CUBES
    PERC_OVERLAP = PERC_OVERLAP
    
    # create instance of clustering alg
    cluster_alg = sklearn.cluster.DBSCAN(eps=DBSCAN_EPSILON, min_samples=DBSCAN_MIN_SAMPLES, metric='euclidean')

    # Instantiate kepler mapper object
    mapper = km.KeplerMapper(verbose=0)
    
    # defining filter function as projection on to the first 2 component axis
    pca = PCA(n_components=1)
    lens = pca.fit_transform(X)
    #pca.fit_transform(X)
    principle_component = max(abs(pca.components_[0].min()), abs(pca.components_[0].max()))
    max_index = 0
    for i in range(len(pca.components_[0])):
        if abs(pca.components_[0][i]) == principle_component:
            max_index = i
    print("Primary variable: ", continuous_variables[max_index])
    print("Corresponding component: ", pca.components_[0][max_index])
    print('Explained Variance: ', pca.explained_variance_ratio_)
    #lens = np.array(X[continuous_variables[max_index]])
    #lens = np.array(X[['PredictedTN', 'PredictedSS']])
    # Generate the simplicial complex
    scomplex = mapper.map(lens, X, cover=km.Cover(n_cubes=N_CUBES, perc_overlap=PERC_OVERLAP), clusterer=cluster_alg,
                         remove_duplicate_nodes=True)  
    summary_variable = mapper.project(np.array(X), projection=projected_var_indices, scaler=None)

    pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

    color_values = lens[:,0] - lens[:,0].min()
    #color_function_name = [continuous_variables[max_index]]
    color_function_name = ['Principle Component: ' + continuous_variables[max_index]]
    my_colorscale = pl_brewer
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                     color_values=color_values,  
                                                                     color_function_name=color_function_name, 
                                                                     colorscale=my_colorscale)

    bgcolor = 'rgba(10,10,10, 0.9)'
    # y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

    plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                     factor_size=2.5, edge_linewidth=0.5)
    plot_title = str(keys[0]) + str(DBSCAN_EPSILON) + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    # plot_title = 'Pool 13, Summer 1993-1999; Epsilon ' + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    layout = plot_layout(title=plot_title,  
                         width=620, height=570,
                         annotation_text=get_kmgraph_meta(mapper_summary),  
                         bgcolor=bgcolor)

    # FigureWidget is responsible for event listeners

    fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
    fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
    fw_summary = summary_fig(mapper_summary, height=300)

    dashboard = hovering_widgets(kmgraph, 
                                 fw_graph, 
                                 bgcolor=bgcolor, 
                                 member_textbox_width=600)

    # DESIRED FILE PATH, CHANGE TO FIT YOUR LOCAL MACHINE
    directory_path = "Users\\killiad\\Documents\\Senior\\REU\\Mapper\\Backwater\\"
    
    #Update the fw_graph colorbar, setting its title:
    fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'
    #html_output_path = directory_path + str(keys[0]) + 'PCA_2' + 'all_var_' + 'Eps_' + str(DBSCAN_EPSILON) +'MinS_' + str(DBSCAN_MIN_SAMPLES) + 'NCUBES_' + str(N_CUBES[0]) + 'PEROvLp_' + str(PERC_OVERLAP[0]) + '.html'
    html_output_path = directory_path + str(keys[0]) + 'PCA_1' + 'all_var_' + 'Eps_' + str(DBSCAN_EPSILON) +'MinS_' + str(DBSCAN_MIN_SAMPLES) + 'NCUBES_' + str(N_CUBES) + 'PEROvLp_' + str(PERC_OVERLAP) + '.html'
    html_output_path = html_output_path.replace(" ", "-")
    html_output_path = html_output_path.replace(":", "_")
    html_output_path = "C:\\" + html_output_path
    mapper.visualize(scomplex, color_values=color_values, color_function_name=color_function_name, path_html=html_output_path,
                    lens = summary_variable, lens_names = projected_vars)
    return scomplex, X



def mapper(df, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = [7,7], PERC_OVERLAP = [.5,.5]):
    """
    str(df_stratum_season_time_dict_list[1].keys())
    """

    if len(df) == 0:
        print("Not Enough Data")
        return
    
    df = df[["WDP", "SECCHI", "TEMP", "DO", 
           "TURB", "VEL", "TP", 
           "TN", "SS", "CHLcal"]]
    df.reset_index(drop = True, inplace = True)
    X = RobustScaler().fit_transform(df)
    
    continuous_variables = ["WDP", "SECCHI", "TEMP", "DO", 
           "TURB", "VEL", "TP", 
           "TN", "SS", "CHLcal"]
    var_to_index = {continuous_variables[i] : i for i in range(len(continuous_variables))}
    
    projected_vars = continuous_variables
    #projected_vars = ['PredictedTP, PredictedTN']
    projected_var_indices = [var_to_index[var] for var in projected_vars]
    
    if X.shape[0]<10:
        #print(X)
        print("Not enough data in ", keys, "_size = ", X.shape[0])
        return(X.shape[0], -1)

    # defining clustering and kmapper parameters
    DBSCAN_EPSILON = DBSCAN_EPSILON
    DBSCAN_MIN_SAMPLES = DBSCAN_MIN_SAMPLES
    N_CUBES = N_CUBES
    PERC_OVERLAP = PERC_OVERLAP
    
    # create instance of clustering alg
    cluster_alg = sklearn.cluster.DBSCAN(eps=DBSCAN_EPSILON, min_samples=DBSCAN_MIN_SAMPLES, metric='euclidean')

    # Instantiate kepler mapper object
    mapper = km.KeplerMapper(verbose=0)
    
    # defining filter function as projection on to the first 2 component axis
    pca = PCA(n_components=2)
    lens = pca.fit_transform(X)
    #pca.fit_transform(X)
    principle_component = max(abs(pca.components_[0].min()), abs(pca.components_[0].max()))
    max_index = 0
    for i in range(len(pca.components_[0])):
        if abs(pca.components_[0][i]) == principle_component:
            max_index = i
    print("Primary variable: ", continuous_variables[max_index])
    print("Corresponding component: ", pca.components_[0][max_index])
    print('Explained Variance: ', pca.explained_variance_ratio_)
    #lens = np.array(X[continuous_variables[max_index]])
    #lens = np.array(X[['PredictedTN', 'PredictedSS']])
    # Generate the simplicial complex
    scomplex = mapper.map(lens, X, cover=km.Cover(n_cubes=N_CUBES, perc_overlap=PERC_OVERLAP), clusterer=cluster_alg,
                         remove_duplicate_nodes=True)  
    summary_variable = mapper.project(np.array(X), projection=projected_var_indices, scaler=None)

    pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

    color_vals = water_df[continuous_variables]
    color_function_names = continuous_variables
    my_colorscale = pl_brewer
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                     color_values=color_values,  
                                                                     color_function_name=color_function_name, 
                                                                     colorscale=my_colorscale)

    bgcolor = 'rgba(10,10,10, 0.9)'
    # y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

    plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                     factor_size=2.5, edge_linewidth=0.5)
    plot_title = "LaGrange" + str(DBSCAN_EPSILON) + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    # plot_title = 'Pool 13, Summer 1993-1999; Epsilon ' + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    layout = plot_layout(title=plot_title,  
                         width=620, height=570,
                         annotation_text=get_kmgraph_meta(mapper_summary),  
                         bgcolor=bgcolor)

    # FigureWidget is responsible for event listeners

    fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
    fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
    fw_summary = summary_fig(mapper_summary, height=300)

    dashboard = hovering_widgets(kmgraph, 
                                 fw_graph, 
                                 bgcolor=bgcolor, 
                                 member_textbox_width=600)

    # DESIRED FILE PATH, CHANGE TO FIT YOUR LOCAL MACHINE
    directory_path = "Users\\killiad\\Documents\\Senior\\REU\\Mapper\\LaGrange\\"
    
    #Update the fw_graph colorbar, setting its title:
    fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'
    #html_output_path = directory_path + str(keys[0]) + 'PCA_2' + 'all_var_' + 'Eps_' + str(DBSCAN_EPSILON) +'MinS_' + str(DBSCAN_MIN_SAMPLES) + 'NCUBES_' + str(N_CUBES[0]) + 'PEROvLp_' + str(PERC_OVERLAP[0]) + '.html'
    html_output_path = directory_path + "LaGrange_Scaled_" + 'PCA_1' + 'all_var_' + 'Eps_' + str(DBSCAN_EPSILON) +'MinS_' + str(DBSCAN_MIN_SAMPLES) + 'NCUBES_' + str(N_CUBES) + 'PEROvLp_' + str(PERC_OVERLAP) + '.html'
    html_output_path = html_output_path.replace(" ", "-")
    html_output_path = html_output_path.replace(":", "_")
    html_output_path = "C:\\" + html_output_path
    mapper.visualize(scomplex, color_values=color_values, color_function_name=color_function_name, path_html=html_output_path,
                    lens = summary_variable, lens_names = projected_vars)
    return scomplex, df

In [ ]:
dbclus_dict = {}
for i in df_stratum_season_time_dict_list:
    dbclus_dict[str(list(i.keys()))]=cluster_fun(i)

In [ ]:
mapper_pca_output_dict = {}
#for i in df_stratum_season_time_dict_list:
#    mapper_pca_output_dict[str(list(i.keys()))] = mapper_pca_func(i)

In [ ]:
n_cubes = [10]
perc_overlap = [.35]
mapper_pca_output_dict = {}
mapper_pca_output_df = {}
for cubes in n_cubes:
    for perc in perc_overlap:
        for i in df_stratum_season_time_dict_list:
            scomplex, X = mapper_pca_func(i, N_CUBES = cubes, PERC_OVERLAP = perc)
            if type(X) != int:
                mapper_pca_output_dict[str(list(i.keys()))] = scomplex
                mapper_pca_output_df[str(list(i.keys()))] = X

In [21]:
mapper_pca_output_dict = {}
mapper_pca_output_df = {}
scomplex, X = mapper(predicted_df, N_CUBES = [50,50], PERC_OVERLAP = [.35,.35], DBSCAN_MIN_SAMPLES = 10, DBSCAN_EPSILON = 1)
if type(X) != int:
    mapper_pca_output_dict['LaGrange'] = scomplex
    mapper_pca_output_df['LaGrange'] = X

Primary variable:  SS
Corresponding component:  0.6402743093586369
Explained Variance:  [0.44654951 0.18992681]


In [22]:
import json


json = json.dumps(mapper_pca_output_dict)
f = open(r"C:\Users\killiad\Documents\Senior\REU\Mapper\LaGrange\LaGrange_Scaled.json","w")
f.write(json)
f.close()

In [11]:
predicted_df.head()

,SHEETBAR,DATE,LATITUDE,LONGITUDE,FLDNUM,STRATUM,LOCATCD,TN,TP,TEMP,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
64787,46000030,07/12/1993,40.553983,-89.673876,"Havana, IL",1,9362003,6.311,0.239,26.2,...,0,0,0,0,0,0,0,0,0,0
64788,46000031,07/12/1993,40.567896,-89.656652,"Havana, IL",1,9362002,4.895,0.258,27.0,...,0,0,0,0,0,0,0,0,0,0
64789,46000032,07/12/1993,40.578620,-89.653755,"Havana, IL",1,9362001,5.947,0.246,26.3,...,0,0,0,0,0,0,0,0,0,0
64790,46000038,07/12/1993,40.526434,-89.847580,"Havana, IL",2,9362020,6.204,0.249,26.9,...,0,0,0,0,0,0,0,0,0,0
64791,46000040,07/12/1993,40.551597,-89.780275,"Havana, IL",1,9362004,2.833,0.258,26.8,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
mapper_pca_output_dict["['Stratum 1 WINTER 93-00: ']"]['nodes']['cube2_cluster0']

In [23]:
import pickle as pk
pickle = pk.dump(mapper_pca_output_df, open(r"C:\Users\killiad\Documents\Senior\REU\Mapper\LaGrange\LaGrange_Scaled.p", "wb"))